Code to access the IDR and perform autocorrelation analysis

In [ ]:
from IPython import get_ipython
import warnings
import omero
from idr import connection

import requests
from pandas import DataFrame
from pandas import read_csv
from pandas import concat
from io import StringIO

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

Access database and build dataframe of image ID

In [ ]:
def getBulkAnnotationAsDf(screenID, conn):
    sc = conn.getObject('Screen', screenID)
    for ann in sc.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            if (ann.getFile().getName() == 'bulk_annotations'):
                number = ann.getFile().getSize()
                ofId = ann.getFile().getId()
                break

    original_file = omero.model.OriginalFileI(ofId, False)

    table = conn.c.sf.sharedResources().openTable(original_file)
    try:
        rowCount = table.getNumberOfRows()

        column_names = [col.name for col in table.getHeaders()]

        black_list = []
        column_indices = []
        for column_name in column_names:
            if column_name in black_list:
                continue
            column_indices.append(column_names.index(column_name))

        table_data = table.slice(column_indices, None)
    finally:
        table.close()

    data = []
    for index in range(rowCount):
        row_values = [column.values[index] for column in table_data.columns]
        data.append(row_values)

    dfAnn = DataFrame(data)
    dfAnn.columns = column_names
    return dfAnn

#connect to database
conn = connection('idr.openmicroscopy.org')

#get annotations for screen
df = getBulkAnnotationAsDf(2651, conn)

#close the connection
conn.close()

# extract images that contain T53BP1
subDF = df[df['Channels'].str.contains('TP53BP1')]

Build Image ID dataframe

In [ ]:
# extract HepG2 cell images
HepG2DF = subDF[subDF['Characteristics [Cell Line]'].str.contains('HepG2')]
HepG2DF=HepG2DF.drop(HepG2DF.columns[[2,3,4,6,7,10,13,14,15,16,17,18,19,20,21,22]], axis=1)
HepG2DF["image_id"] = ""
HepG2DF["DA"] = np.nan
HepG2DF["stdev"] = np.nan
HepG2DF["count"] = np.nan
# get plates
plates = HepG2DF['Plate'].unique()

from IPython.display import display, HTML
import requests
# initial data
IDR_BASE_URL = "https://idr.openmicroscopy.org"

INDEX_PAGE = "%s/webclient/?experimenter=-1" % IDR_BASE_URL
# create http session
with requests.Session() as session:
    request = requests.Request('GET', INDEX_PAGE)
    prepped = session.prepare_request(request)
    response = session.send(prepped)
    if response.status_code != 200:
        response.raise_for_status()
        
        
# get image id number
image_id = []
well_id = []
for i in range(len(plates)):
    WELLS_IMAGES_URL = "{base}/webgateway/plate/{plate_id}/{field}/"
    plate_id = plates[i]
    qs = {'base': IDR_BASE_URL, 'plate_id': plate_id, 'field': 0}
    url = WELLS_IMAGES_URL.format(**qs)
    grid = session.get(url).json()
    rowlabels = grid['rowlabels']
    collabels = grid['collabels']
    for row in grid['grid']:
        for cell in row:
            if cell is not None:
                image2=cell['id']
                well2=cell['wellId']
                image_id.append(image2)
                well_id.append(well2)

#finalize DataFrame
for i in range(len(image_id)):
    HepG2DF.loc[HepG2DF.Well == well_id[i],'image_id'] = image_id[i]
    
HepG2DF.to_csv("HepG2Data.csv")




Perform autocorrelation analysis

In [ ]:
# set up for ICS analysis
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import shutil

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D

from PIL import Image
import skimage.measure 

import scipy.optimize as opt
from scipy.fftpack import fft2, ifft2

np.random.seed(6)
lbl_cmap = random_label_cmap()

current_directory = os.getcwd()

# set up functions

def autocorrelation(x,intensity) :
    f = fft2(x)
    fConj = np.conj(f)
    p = f*fConj
    pi = ifft2(p)
    return np.fft.fftshift(np.real(pi)/((intensity**2)*len(x)*len(x[0])))-1

def correlationCorrection(x,corr) : 
    x[x < 0] = 0
    y  = x*(1/corr)*9 
    return y

def func(M,a,b,c) :
    x0 = xDim[indFitCrop[0]] # x and y values at peak
    y0 = yDim[indFitCrop[1]]
    x,y=M
    return a*np.exp(-((x-x0)**2 + (y-y0)**2)/c**2)+b

def fitCropping(z,h) :
    ind = np.unravel_index(np.argmax(z, axis=None), z.shape)
    if ind[0] > z.shape[0]-h or ind[1] > z.shape[1]-h or ind[0] < h or ind[1] < h:
        return np.zeros((2*h,2*h))
    else:
        return z[ind[0]-h:ind[0]+h,ind[1]-h:ind[1]+h]
    
# define constants
pixelSize = 0.64
cropDim = 10

#filters for incorrect fits
areaMax = 3000 
areaMin = 300 
intMax = 3500
intMin = 200
w0Max = 3
w0Min = 0
errorLimit = 5
nucleiMinCount = 50

#constants for fitting
xDim = np.linspace(-(cropDim-1),cropDim,cropDim*2)*pixelSize
yDim = np.linspace(-(cropDim-1),cropDim,cropDim*2)*pixelSize
X,Y=np.meshgrid(xDim,yDim)
xdata = np.vstack((X.ravel(), Y.ravel()))
initialGuess = (0.8, 0.14, 1.4)

model = StarDist2D.from_pretrained('2D_demo')

In [ ]:
conn = connection('idr.openmicroscopy.org')

In [ ]:
#load image
for i in range(19357, len(HepG2DF.index)):
    imageID = HepG2DF.iloc[i]['image_id']
    image = conn.getObject("Image", imageID)
    pixels = image.getPrimaryPixels()
    dapiMatrix = pixels.getPlane(0, 0, 0)
    AbMatrix = pixels.getPlane(0,1,0)
    
    # run analysis
    n_channel = 1 if dapiMatrix.ndim == 2 else dapiMatrix.shape[-1]
    axis_norm = (0,1)   # normalize channels independently

    #segment DAPI
    img = normalize(dapiMatrix, 1,99.8, axis=axis_norm) # dapiMatrix[i]
    labels, details = model.predict_instances(img, prob_thresh=0.8)
    
    # subtract image backgrounds
    AbBack = np.percentile(AbMatrix,5) # 5th percentile of image  #AbMatrix[i]
    AbImage = np.subtract(AbMatrix,AbBack) #AbMatrix[i]
    AbImage[AbImage<0] = 0

    #area, ab intensity, bounding box, extent=corrFactor
    regions = skimage.measure.regionprops_table(labels,intensity_image=AbImage,properties=('area','intensity_mean','bbox','extent','image'))
    nuclei=pd.DataFrame.from_dict(regions)
    
    #initialize data lists
    DAList = []

    for k in range(len(nuclei)):
    
        #get bounding boxes
        rowMin = nuclei['bbox-0'][k]
        colMin = nuclei['bbox-1'][k]
        rowMax = nuclei['bbox-2'][k]
        colMax = nuclei['bbox-3'][k]
        padRow = rowMax-rowMin
        padCol = colMax-colMin
        
        #Ab speciific
        meanInt = np.rint(nuclei.intensity_mean[k])
    
        crop = AbImage[rowMin:rowMax, colMin:colMax]
        cropAve = crop*nuclei.image[k]
        cropAve[cropAve == 0] = meanInt
        cropPad = np.pad(cropAve,((padRow,padRow),(padCol,padCol)), 'constant',constant_values=meanInt)
    
        try:
            autoCorr = autocorrelation(cropPad,meanInt) # run correlation
            autoCorrCorrected = correlationCorrection(autoCorr,nuclei.extent[k]) # correct correlation
        
            fitCrop = fitCropping(autoCorrCorrected,cropDim) # crop autocorrelation for fitting
            indFitCrop = np.unravel_index(np.argmax(fitCrop, axis=None), fitCrop.shape)#find peak in cropped image
            popt,pcov = opt.curve_fit(func,xdata,fitCrop.ravel(), p0 = initialGuess) # fit
        
            # pull out parameters
            perr = np.sqrt(np.diag(pcov)) # stdev of fits
            relError = perr/popt*100 # percent deviation from measurement
            nop = 1/popt[0]
            w0 = popt[2]
            DA = meanInt/nop
            relErrorMax = max(relError[0],relError[2]) #maximum percentDev between g(0,0) and w0
            area = nuclei.area[k]
            
            # filter to remove incorrect results
            if area > areaMax or area < areaMin or w0 > w0Max or w0 < w0Min\
            or relErrorMax > errorLimit or len(nuclei) < nucleiMinCount:
                DA = np.nan
                
        except Exception:
            DA = np.nan

        # add parameters to list
        DAList.append(DA)
        
############## end of nuclear analysis
    
    #calculate mean and stdev of DA
    DAmean=np.nanmean(DAList)
    DAstd=np.nanstd(DAList)
    DAlimit=DAmean+2*DAstd
    replace = np.nan
    DAedited = [replace if ele > DAlimit else ele for ele in DAList]
    DAfinal=np.nanmean(DAedited)
    DAfinalStd=np.nanstd(DAedited)

    HepG2DF.loc[HepG2DF.image_id == imageID,'DA'] = DAfinal
    HepG2DF.loc[HepG2DF.image_id == imageID,'stdev'] = DAfinalStd
    HepG2DF.loc[HepG2DF.image_id == imageID,'count'] = len(nuclei)
    
    percent=i/len(HepG2DF.index)*100
    print(percent)

conn.close()